# Unsupervised Learning
## DSSP Team
## Summer 2020

## PCA

We will consider a _classical_ dataset __decathlon__. It contains the performance of several athletes during two decathlons in 2004 the Decastar and the Olympic Game. Each observation consists of

- the 10 raw performance in the 10 events,
- the ranking in the event,
- the total number of points,
- the name of the event.

We will mainly focus on the 10 first columns corresponding to the raw athletic performance but will also use the total number of points in some plots.

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_style('whitegrid')
import plotly.graph_objects as go 
import plotly.io as pio
#pio.renderers.default = 'notebook'
from sklearn import decomposition #PCA
from adjustText import adjust_text #text labels placement
from scipy.linalg import sqrtm #square root of a matrix

In [ ]:
decathlon = pd.read_csv('../data/decathlon.txt', sep='\t')
decathlon

In [ ]:
decathlon.describe()

### Pairwise analysis

__1)__ Use `seaborn` to plot the two first coordinates (__100m__ and __Long.jump__) adding the __Points__ information as a color as well as a regression line.

__Hint:__ You can use the `'viridis'` palette to have better colors and `adjust_text` to have a better text placement.

In [ ]:
#solution
sns.regplot(data=decathlon, x='100m', y='Long.jump', scatter=False);
sns.scatterplot(data=decathlon, x='100m', y='Long.jump', hue='Points',
palette='viridis')
texts = [
    plt.text(x, y, s, fontsize=8, horizontalalignment='center', verticalalignment='bottom')
    for x, y, s in zip(decathlon['100m'], decathlon['Long.jump'], decathlon.index)]
adjust_text(texts, arrowprops=dict(arrowstyle='->', color='k'));

__2)__ Do you think those values are correlated?

In [ ]:
#solution
# yes!

__3)__ Compute the correlation matrix between the 10 first variables and display it use a heatmap.

In [ ]:
#solution
decathlon.iloc[:,0:10].corr()

In [ ]:
#solution
sns.heatmap(np.abs(decathlon.iloc[:,0:10].corr()));

__4)__ Can you guess which are the most correlated variables and the least ones?

In [ ]:
#solution
#Most correlated: 100.m and Long.jump, Discus and Shot.put..
#Least correlated: Pole.vault and 100m, 100m and 1500m

__5)__ Confirm those findings by looking at the pairwise scatterplot.

In [ ]:
#solution
sns.pairplot(decathlon.iloc[:,0:10]);

In [ ]:
#solution
sns.pairplot(decathlon.iloc[:, [0, 2, 5,9]]);

### 3d plot

We are now ready to use `plotly` to visualize the dataset in 3d.

A 3d scatter plot can be obtained with the following code: 

In [ ]:
scat = go.Scatter3d(x=decathlon['100m'], y=decathlon['Long.jump'], z=decathlon['Shot.put'],
    mode='markers',
    marker=dict(color=decathlon['Points'],
        colorscale='viridis',
        showscale=True))
fig = go.Figure()
fig.add_trace(scat)
fig.update_layout(scene=dict(camera=dict(projection=dict(type='orthographic')),
    xaxis=dict(showspikes=False),
    yaxis=dict(showspikes=False),
    zaxis=dict(showspikes=False),
    )
    )
fig.update_traces()
fig.show()

Note that we have used a orthographic projection rather than a perspective one.

__6)__ Play with this interactive graphic to find he projection that appears to lose the less information?

__7)__ This task is similar to the PCA where we look a the subspace that minimizes the error between the data and its projection. Draw the ellipse corresponding to the eigenvectors and the eigenvalues of the covariance matrix to verify this.

__Hint:__ Use the `Mesh3d` layer of `plotly` with the `alphahull=0` option and the following ellipse generating code:

In [ ]:
def sphere():
    theta = np.arange(-np.pi / 2, np.pi / 2, 0.1)
    phi = np.arange(0, 2 * np.pi, 0.1)
    x = np.cos(theta[:, None]) * np.cos(phi[None, :])
    y = np.cos(theta[:, None]) * np.sin(phi[None ,:])
    z = np.sin(theta[:, None]) * np.ones(phi.shape)[None, :]
    return(np.vstack((x.flatten(), y.flatten(), z.flatten())).T)


In [ ]:
def ellipse3d(mean, cov):
    return(1.95 * sphere().dot(sqrtm(cov))\
        + mean[None, :])

In [ ]:
#solution
ellipse = ellipse3d(decathlon.iloc[:, 0:3].mean(), decathlon.iloc[:, 0:3].cov())
scat = go.Scatter3d(x=decathlon['100m'], y=decathlon['Long.jump'], z=decathlon['Shot.put'],
    mode='markers',
    marker=dict(color=decathlon['Points'],
        colorscale='viridis',
        showscale=True))
ell = go.Mesh3d(x=ellipse[:, 0], y=ellipse[:, 1], z=ellipse[:, 2],
    opacity=.1, alphahull=0)
fig = go.Figure()
fig.add_trace(scat)
fig.add_trace(ell)
fig.update_layout(scene=dict(camera=dict(projection=dict(type='orthographic')),
    xaxis=dict(showspikes=False),
    yaxis=dict(showspikes=False),
    zaxis=dict(showspikes=False),
    ))
fig.update_traces()
fig.show()

__8)__ Can you verify that the best subspace is defined by the direction of the eigenvectors with the largest eigenvalues?

### Scaling?

So far, we have use the raw performance. That is we compare duration with length as well as 100m time with 1500m time. Obviously, this is not a good idea. All those values should be measured in a common scale. The most classical technique is to _normalize_ by substracting its mean and dividing by the standard deviation. After doing thiw, we may repeat all the previous experiments with those rescaled data.

__9)__ Make a copy of the decathlon dataframe and normalize the 10 first columns of this new data frame.

__Hint:__ Use `apply` and a lambda function. 

In [ ]:
#solution
decathlon_scaled = decathlon.copy()
decathlon_scaled.iloc[:, 0:10] = decathlon_scaled.iloc[:, 0:10].\
    apply(lambda x: (x - np.mean(x)) / np.std(x))

In [ ]:
#solution
decathlon_scaled

__10)__ Plot the corresponding scatter plot and compare it with the one obtained with the raw data.

In [ ]:
#solution
sns.regplot(data=decathlon_scaled, x='100m', y='Long.jump', scatter=False);
sns.scatterplot(data=decathlon_scaled, x='100m', y='Long.jump', hue='Points')
texts = [
    plt.text(x, y, s, fontsize=8, horizontalalignment='center', verticalalignment='bottom')
    for x, y, s in zip(decathlon_scaled['100m'], decathlon_scaled['Long.jump'], decathlon_scaled.index)]
adjust_text(texts, arrowprops=dict(arrowstyle='->', color='k'));

In [ ]:
#solution
sns.regplot(data=decathlon, x='100m', y='Long.jump', scatter=False);
sns.scatterplot(data=decathlon, x='100m', y='Long.jump', hue='Points')
texts = [
    plt.text(x, y, s, fontsize=8, horizontalalignment='center', verticalalignment='bottom')
    for x, y, s in zip(decathlon['100m'], decathlon['Long.jump'], decathlon.index)]
adjust_text(texts, arrowprops=dict(arrowstyle='->', color='k'));

__11)__ What if one imposes _equal_ axis?

__Hint:__ Use `plt.axis('equal')`.

In [ ]:
#solution
sns.regplot(data=decathlon, x='100m', y='Long.jump', scatter=False);
sns.scatterplot(data=decathlon, x='100m', y='Long.jump', hue='Points')
texts = [
    plt.text(x, y, s, fontsize=8, horizontalalignment='center', verticalalignment='bottom')
    for x, y, s in zip(decathlon['100m'], decathlon['Long.jump'], decathlon.index)]
adjust_text(texts, arrowprops=dict(arrowstyle='->', color='k'))
plt.axis('equal');

In [ ]:
#solution
sns.regplot(data=decathlon_scaled, x='100m', y='Long.jump', scatter=False);
sns.scatterplot(data=decathlon_scaled, x='100m', y='Long.jump', hue='Points')
texts = [
    plt.text(x, y, s, fontsize=8, horizontalalignment='center', verticalalignment='bottom')
    for x, y, s in zip(decathlon_scaled['100m'], decathlon_scaled['Long.jump'], decathlon_scaled.index)]
adjust_text(texts, arrowprops=dict(arrowstyle='->', color='k'))
plt.axis('equal');

__12)__ Repeat the same experiment replacing the `100m` by the `1500m`.

In [ ]:
#solution
sns.regplot(data=decathlon, x='1500m', y='Long.jump', scatter=False);
sns.scatterplot(data=decathlon, x='1500m', y='Long.jump', hue='Points')
texts = [
    plt.text(x, y, s, fontsize=8, horizontalalignment='center', verticalalignment='bottom')
    for x, y, s in zip(decathlon['1500m'], decathlon['Long.jump'], decathlon.index)]
adjust_text(texts, arrowprops=dict(arrowstyle='->', color='k'));

In [ ]:
#solution
sns.regplot(data=decathlon_scaled, x='1500m', y='Long.jump', scatter=False);
sns.scatterplot(data=decathlon_scaled, x='1500m', y='Long.jump', hue='Points')
texts = [
    plt.text(x, y, s, fontsize=8, horizontalalignment='center', verticalalignment='bottom')
    for x, y, s in zip(decathlon_scaled['1500m'], decathlon_scaled['Long.jump'], decathlon_scaled.index)]
adjust_text(texts, arrowprops=dict(arrowstyle='->', color='k'));

In [ ]:
#solution
sns.regplot(data=decathlon, x='1500m', y='Long.jump', scatter=False);
sns.scatterplot(data=decathlon, x='1500m', y='Long.jump', hue='Points')
texts = [
    plt.text(x, y, s, fontsize=8, horizontalalignment='center', verticalalignment='bottom')
    for x, y, s in zip(decathlon['1500m'], decathlon['Long.jump'], decathlon.index)]
adjust_text(texts, arrowprops=dict(arrowstyle='->', color='k'))
plt.axis('equal');

In [ ]:
#solution
sns.regplot(data=decathlon_scaled, x='1500m', y='Long.jump', scatter=False);
sns.scatterplot(data=decathlon_scaled, x='1500m', y='Long.jump', hue='Points')
texts = [
    plt.text(x, y, s, fontsize=8, horizontalalignment='center', verticalalignment='bottom')
    for x, y, s in zip(decathlon_scaled['1500m'], decathlon_scaled['Long.jump'], decathlon_scaled.index)]
adjust_text(texts, arrowprops=dict(arrowstyle='->', color='k'))
plt.axis('equal');

__13)__ What about the correlation matrix, the pairwise scatterplots and the 3d plot?

In [ ]:
#solution
decathlon_scaled.iloc[:,1:10].corr()

In [ ]:
#solution
sns.pairplot(decathlon_scaled.iloc[:,0:10]);

In [ ]:
#solution
ellipse = ellipse3d(decathlon_scaled.iloc[:, 0:3].mean(), decathlon_scaled.iloc[:, 0:3].cov())
scat = go.Scatter3d(x=decathlon_scaled['100m'], y=decathlon_scaled['Long.jump'], z=decathlon_scaled['Shot.put'],
    mode='markers',
    marker=dict(color=decathlon_scaled['Points'],
        colorscale='viridis',
        showscale=True))
ell = go.Mesh3d(x=ellipse[:, 0], y=ellipse[:, 1], z=ellipse[:, 2],
    opacity=.1, alphahull=0)
fig = go.Figure()
fig.add_trace(scat)
fig.add_trace(ell)
fig.update_layout(scene=dict(camera=dict(projection=dict(type='orthographic')),
    xaxis=dict(showspikes=False),
    yaxis=dict(showspikes=False),
    zaxis=dict(showspikes=False),
    ))
fig.update_traces()
fig.show()

### Principal Component Analysis

The library `scikit-learn` contains a PCA function that we are going to use.

In [ ]:
pca_scaled = decomposition.PCA(n_components=10).fit(decathlon_scaled.iloc[:, 0:10])
decathlon_scaled_pca = pca_scaled.transform(decathlon_scaled.iloc[:, 0:10])

`pca_scaled` contains the all the information computed by the PCA (for instance the eigenvalues in the field `singular_values_`, the change of basis matrix in `components_`) and `decathlon_scaled_pca` contains the coordinates of `decathlon_scaled` in the PCA basis.

__14)__ Verify the the coordinates in the PCA basis can be obtained by a product between the coordinates in the old basis and the change of basis matrix

In [ ]:
#solution
np.max(np.abs(decathlon_scaled_pca - decathlon_scaled.iloc[:, 0:10].values.dot(pca_scaled.components_.T)))

__15)__ Plot the points in the new coordinates

In [ ]:
#solution
decathlon_scaled_pca_df = pd.DataFrame({'X': decathlon_scaled_pca[:, 0],
    'Y': decathlon_scaled_pca[:, 1]},
    index = decathlon.index)
decathlon_scaled_pca_df['Points'] = decathlon['Points']
sns.scatterplot(data=decathlon_scaled_pca_df, x='X', y='Y', hue='Points', palette='viridis')
texts = [
    plt.text(x, y, s, fontsize=8, horizontalalignment='center', verticalalignment='bottom')
    for x, y, s in zip(decathlon_scaled_pca_df['X'], decathlon_scaled_pca_df['Y'], decathlon_scaled_pca_df.index)]
adjust_text(texts, arrowprops=dict(arrowstyle='->', color='k'))
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
plt.axis('equal');

__16)__ In statistics, one often looks at the correlation between the new axis and the old ones. This can be computed in Python with

In [ ]:
decathlon_scaled_pca_nor = decathlon_scaled_pca/np.sqrt((decathlon_scaled_pca ** 2).sum(axis=0))
decathlon_scaled_nor = decathlon_scaled.iloc[:, 0:10]/np.sqrt((decathlon_scaled.iloc[:, 0:10] ** 2).sum(axis=0))
decathlon_corr_circle = decathlon_scaled_pca_nor.T.dot(decathlon_scaled_nor)
decathlon_corr_circle

Note that by construction the norm of the correlation vector for a given original axis is 1.

In [ ]:
(decathlon_corr_circle ** 2).sum(axis=0)

Plot the correlation of the original axes with the two first new components

In [ ]:
#solution
pcs = decathlon_corr_circle
fig, ax = plt.subplots()
for i, (x, y) in enumerate(zip(pcs[0, :], pcs[1, :])):
    plt.arrow(0, 0, x, y, head_width=.05, color='k')
texts = [ 
    plt.text(x, y, decathlon_scaled.columns[i])
    for i, (x, y) in enumerate(zip(pcs[0, :], pcs[1, :]))
]
#adjust_text(texts, arrowprops=dict(arrowstyle='->', color='k'))
plt.plot([-1, 1], [0, 0], color='grey', ls='--')
plt.plot([0, 0], [-1, 1], color='grey', ls='--')
plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.axis('equal')
plt.grid(b=None)
plt.gca().axison = False
circle = plt.Circle((0, 0), 1, facecolor='none', edgecolor='grey')
ax.add_patch(circle);

__17)__ Which variables are well captured? Can you interpret the new axes (the horizontal and the vertical ones)? 
Can you explain why the long jump appears to be the opposite of the 100m?

In [ ]:
#solution
decathlon_mod = decathlon.copy()
decathlon_mod['100m'] = 100 / decathlon_mod['100m'] 
decathlon_mod['400m'] = 400 / decathlon_mod['400m']
decathlon_mod['110m.hurdle'] = 110 / decathlon_mod['110m.hurdle']
decathlon_mod['1500m'] = 1500 / decathlon_mod['1500m'] 
decathlon_mod_scaled = decathlon_mod.copy()
decathlon_mod_scaled.iloc[:, 0:10] = decathlon_mod_scaled.iloc[:, 0:10].\
    apply(lambda x: (x - np.mean(x)) / np.std(x))
pca_scaled = decomposition.PCA(n_components=10).fit(decathlon_mod_scaled.iloc[:, 0:10])
decathlon_mod_scaled_pca = pca_scaled.transform(decathlon_mod_scaled.iloc[:, 0:10])
decathlon_mod_scaled_pca_nor = decathlon_mod_scaled_pca/np.sqrt((decathlon_mod_scaled_pca ** 2).sum(axis=0))
decathlon_mod_scaled_nor = decathlon_mod_scaled.iloc[:, 0:10]/np.sqrt((decathlon_mod_scaled.iloc[:, 0:10] ** 2).sum(axis=0))
decathlon_corr_circle = decathlon_mod_scaled_pca_nor.T.dot(decathlon_mod_scaled_nor)
pcs = decathlon_corr_circle
fig, ax = plt.subplots()
for i, (x, y) in enumerate(zip(pcs[0, :], pcs[1, :])):
    plt.arrow(0, 0, x, y, head_width=.05, color='k')
texts = [ 
    plt.text(x, y, decathlon_mod_scaled.columns[i])
    for i, (x, y) in enumerate(zip(pcs[0, :], pcs[1, :]))
]
#adjust_text(texts, arrowprops=dict(arrowstyle='->', color='k'))
plt.plot([-1, 1], [0, 0], color='grey', ls='--')
plt.plot([0, 0], [-1, 1], color='grey', ls='--')
plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.axis('equal')
plt.grid(b=None)
plt.gca().axison = False
circle = plt.Circle((0, 0), 1, facecolor='none', edgecolor='grey')
ax.add_patch(circle);

__18)__ Plot the cumulative percentage of inertia. Do you think using 2 dimensions is enough here?

__Hint:__ The inertia is also called the explained variance.

In [ ]:
#solution
plt.bar(np.arange(1, 11), pca_scaled.explained_variance_ratio_.cumsum())
plt.xlabel('Dim.')
plt.ylabel('Cumlative percentage of inertia');
